DOWNLOAD ALL NECESSARY FILE FROM THE FOLDER

In [1]:
import pywt
from numpy import convolve
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
%matplotlib ipympl

In [2]:
texts0 = [[], []]
texts1 = [[], []]

with open("Print_09_v2_PCG_LV.txt", "r") as file:
  textfile = file.readlines()

for text in textfile:
    text = text.split('\t')
    text[1] = text[1].replace('\n',"")
    texts0[0].append(float(text[0]))
    texts0[1].append(float(text[1]))

with open("Print_13_v2_PCG_RV.txt", "r") as file:
  textfile = file.readlines()

for text in textfile:
    text = text.split('\t')
    text[1] = text[1].replace('\n',"")
    texts1[0].append(float(text[0]))
    texts1[1].append(float(text[1]))


fig0 = plt.figure()
ax0 = fig0.add_axes([0.1,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="ElectroCardioGram Signal")
ax0.plot(texts0[0], texts0[1], label="ECG Signal")
ax0.plot(texts1[0], texts1[1], label="PCG Signal", alpha=0.5)
ax0.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
temporaryconvolvedsignal = texts0 # store temporary data for leveling on DWT

def continuouswavelettransform(var):
    pass

def discretewavelettransfrom(var, waveletfunction, scalingfunction):
    return convolve(var, waveletfunction, "same"), convolve(var, scalingfunction, "same")

def waveletsignalfunction(wavelettype, waveletlength, waveletscale):
    if wavelettype == "DWT-Haar":
        wavelet = pywt.Wavelet("haar")
        _, waveletfunction, x = wavelet.wavefun(level=waveletscale)
        return x, waveletfunction
    elif wavelettype == "DWT-Symlet-4":
        wavelet = pywt.Wavelet("sym4")
        _, waveletfunction, x = wavelet.wavefun(level=waveletscale)
        return x, waveletfunction
    elif wavelettype == "CWT-Morlet":
        wavelet = pywt.ContinuousWavelet("morl")
        waveletfunction, x = wavelet.wavefun(level=waveletscale)
        return x, waveletfunction
    elif wavelettype == "CWT-Mexican Hat":
        wavelet = pywt.ContinuousWavelet("mexh")
        waveletfunction, x = wavelet.wavefun(level=waveletscale)
        return x, waveletfunction

def scalingsignalfunction(wavelettype, waveletlength, waveletscale):
    if wavelettype == "DWT-Haar":
        wavelet = pywt.Wavelet("haar")
        scalingfunction, _, x = wavelet.wavefun(level=waveletscale)
        return x, scalingfunction
    elif wavelettype == "DWT-Symlet-4":
        wavelet = pywt.Wavelet("sym4")
        scalingfunction, _, x = wavelet.wavefun(level=waveletscale)
        return x, scalingfunction

def on_change(change):
    with out:
        global temporaryconvolvedsignal
        temporaryconvolvedsignal = texts0

        fig1.clear()
        ax1 = fig1.add_axes([0.12,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Signal")

        fig2.clear()
        ax2 = fig2.add_axes([0.12,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Transform Signal")

        try:
            xaxis, waveletfunction= waveletsignalfunction(
                selectwavelettype.get_interact_value(), 
                selectwaveletlength.get_interact_value(), 
                selectwaveletscale.get_interact_value())
            ax1.plot(xaxis, waveletfunction, label="Wavelet Function")
            ax1.legend()
            selectwaveletlength.value = len(xaxis)
        except Exception as e:
            pass
        
        try:
            xaxis, scalingfunction= scalingsignalfunction(
                selectwavelettype.get_interact_value(), 
                selectwaveletlength.get_interact_value(), 
                selectwaveletscale.get_interact_value())
            ax1.plot(xaxis, scalingfunction, label="Scaling Function")
            ax1.legend()
            selectwaveletlength.value = len(xaxis)
        except Exception as e:
            pass

        if selectwaveletlength.value >= len(temporaryconvolvedsignal[1]):
            out.clear_output()
            print('Length of the wavelet cannot exceed the input length signal! ({})'.format(len(temporaryconvolvedsignal[1])))
            return

        convolvedwaveletsignal, _ = discretewavelettransfrom(temporaryconvolvedsignal[1], waveletfunction, scalingfunction)
        ax2.plot(temporaryconvolvedsignal[0], convolvedwaveletsignal, label="Convolved Signal")
        out.clear_output()

def on_button_clicked(_):
    global temporaryconvolvedsignal

    with out:
        fig1.clear()
        ax1 = fig1.add_axes([0.12,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Signal")

        fig2.clear()
        ax2 = fig2.add_axes([0.12,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Transform Signal")
        selectwaveletscale.value = selectwaveletscale.get_interact_value()+1

        try:
            xaxis, waveletfunction= waveletsignalfunction(
                selectwavelettype.get_interact_value(), 
                selectwaveletlength.get_interact_value(), 
                selectwaveletscale.get_interact_value())
            ax1.plot(xaxis, waveletfunction, label="Wavelet Function")
            ax1.legend()
            selectwaveletlength.value = len(xaxis)
        except Exception as e:
            pass
        
        try:
            xaxis, scalingfunction= scalingsignalfunction(
                selectwavelettype.get_interact_value(), 
                selectwaveletlength.get_interact_value(), 
                selectwaveletscale.get_interact_value())
            ax1.plot(xaxis, scalingfunction, label="Scaling Function")
            ax1.legend()
            selectwaveletlength.value = len(xaxis)
        except Exception as e:
            pass

        if selectwaveletlength.value >= len(temporaryconvolvedsignal[1]):
            out.clear_output()
            print('Length of the wavelet cannot exceed the input length signal! ({})'.format(len(temporaryconvolvedsignal[1])))
            return
        convolvedwaveletsignal, temporaryconvolvedsignal[1] = discretewavelettransfrom(temporaryconvolvedsignal[1], waveletfunction, scalingfunction)
        ax2.plot(temporaryconvolvedsignal[0], convolvedwaveletsignal, label="Convolved Signal")
        out.clear_output()
        print('Using this button enabling convolution of the scaling function to the signal thus making it increasing in level!')
        

In [4]:
selectwavelettype = widgets.Select(
    options=['DWT-Haar', "DWT-Symlet-4", "CWT-Morlet", "CWT-Mexican Hat"],
    value=None,
    description='Wavelet Type:',
    disabled=False
)

selectwaveletlength = widgets.IntText(
    description='Wavelet Length:',
    disabled=False
)

selectwaveletscale = widgets.FloatSlider(
    value=5,
    min=0,
    max=15.0,
    step=1,
    description='Wavelet Refinement Scale:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

buttonnextwaveletscale = widgets.Button(
    description='Next Wavelet Scale')

out = widgets.Output()

In [5]:
fig1 = plt.figure()
ax1= fig1.add_axes([0.12,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Wavelet Signal")

selectwavelettype.observe(on_change)
selectwaveletlength.observe(on_change)
selectwaveletscale.observe(on_change)
buttonnextwaveletscale.on_click(on_button_clicked)

display(selectwavelettype, selectwaveletlength, selectwaveletscale)

fig2 = plt.figure()
ax2 = fig2.add_axes([0.12,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Transform Signal")
plt.show()

display(buttonnextwaveletscale, out)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Select(description='Wavelet Type:', options=('DWT-Haar', 'DWT-Symlet-4', 'CWT-Morlet', 'CWT-Mexican Hat'), val…

IntText(value=0, description='Wavelet Length:')

FloatSlider(value=5.0, continuous_update=False, description='Wavelet Refinement Scale:', max=15.0, readout_for…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Next Wavelet Scale', style=ButtonStyle())

Output()

In [ ]:
# scale = 200
# delta = 0.2
# dt = 0.001 # 1000 Hz Sampling
# coef, freqs=pywt.cwt(texts1[1], np.arange(1, scale, 4), 'morl')
# X = texts1[0]
# Y = np.linspace(0, np.shape(freqs)[0], np.shape(coef)[0])
# Z = abs(coef)
# X, Y = np.meshgrid(X, Y)

# fig = plt.figure()
# plt.imshow(coef, interpolation="nearest", origin="upper",aspect='auto')
# plt.colorbar()
# plt.show()
# plt.matshow(coef) # doctest: +SKIP
# plt.show() # doctest: +SKIP

In [ ]:
# fig = plt.figure()
# ax = plt.axes(projection='3d', title="Continuous Wavelet Transform")
# plot_cwt = ax.plot_surface(X, Y, Z,rstride=1, cstride=1,
#                 cmap='viridis', edgecolor='none')
# plot_cwt = ax.plot_wireframe(X, Y, Z, rstride=10, cstride=10)

# fig.colorbar(plot_cwt, shrink=0.5, aspect=5)